In [ ]:
#| default_exp inference.anomaly_score_organizer

# Anomaly Score Organizer

> Organize and save images based on their anomaly scores into customizable threshold folders

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import json
import shutil
from pathlib import Path
from typing import Union, List, Dict, Any, Optional
import numpy as np
from tqdm import tqdm

from fastcore.all import *
from fastcore.test import *

In [ ]:
#| export
# Import from existing modules
from vision_ad_tool.inference.prediction_system import (
    predict_image_list_from_file_enhanced,
    predict_image_list
)

from vision_ad_tool.inference.multinode_inference import (
    create_smart_batches,
    scan_folder_structure,
    create_batch_list_file
)

## Core Functions

In [ ]:
#| export
def determine_score_folder(
    anomaly_score: float,  # Anomaly score (0.0 to 1.0)
    score_thresholds: List[float]  # List of score thresholds (e.g., [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
) -> str:  # Returns the folder name based on the score
    """
    Determine which folder an image should go to based on its anomaly score.
    
    The image is placed in the folder corresponding to the smallest threshold that is >= the score.
    
    Example:
        score_thresholds = [0.5, 1.0]
        - score 0.3 -> folder "0.5"
        - score 0.7 -> folder "1.0"
    """
    # Sort thresholds to ensure correct ordering
    sorted_thresholds = sorted(score_thresholds)
    
    # Find the appropriate folder
    for threshold in sorted_thresholds:
        if anomaly_score <= threshold:
            return str(threshold)
    
    # If score exceeds all thresholds, use the last one
    return str(sorted_thresholds[-1])

In [ ]:
#| export
def create_score_folders(
    output_dir: Path,  # Base output directory
    score_thresholds: List[float]  # List of score thresholds
) -> Dict[str, Path]:  # Returns dict mapping threshold strings to folder paths
    """
    Create subdirectories for each score threshold.
    
    Returns a dictionary mapping threshold values to their folder paths.
    """
    output_dir = Path(output_dir)
    folder_map = {}
    
    for threshold in score_thresholds:
        folder_name = str(threshold)
        folder_path = output_dir / folder_name
        folder_path.mkdir(parents=True, exist_ok=True)
        folder_map[folder_name] = folder_path
    
    print(f"✅ Created {len(folder_map)} score folders in {output_dir}")
    for threshold, path in sorted(folder_map.items()):
        print(f"   📁 {threshold}: {path}")
    
    return folder_map

In [ ]:
#| export
def save_image_by_score(
    image_path: Union[str, Path],  # Path to the source image
    anomaly_score: float,  # Anomaly score for the image
    output_dir: Path,  # Base output directory
    score_thresholds: List[float],  # List of score thresholds
    copy_mode: bool = True  # If True, copy files; if False, move files
) -> Path:  # Returns the destination path
    """
    Save (copy or move) an image to the appropriate score folder.
    
    Returns the destination path where the image was saved.
    """
    image_path = Path(image_path)
    
    if not image_path.exists():
        raise FileNotFoundError(f"Image not found: {image_path}")
    
    # Determine target folder
    folder_name = determine_score_folder(anomaly_score, score_thresholds)
    target_folder = output_dir / folder_name
    target_folder.mkdir(parents=True, exist_ok=True)
    
    # Create destination path
    dest_path = target_folder / image_path.name
    
    # Copy or move the file
    if copy_mode:
        shutil.copy2(image_path, dest_path)
    else:
        shutil.move(str(image_path), str(dest_path))
    
    return dest_path

In [ ]:
#| export
def organize_images_by_score(
    prediction_results: List[Dict[str, Any]],  # List of prediction results from predict_image_list
    output_dir: Union[str, Path],  # Base output directory
    score_thresholds: List[float] = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Score thresholds
    copy_mode: bool = True,  # If True, copy files; if False, move files
    save_metadata: bool = True  # If True, save metadata JSON for each folder
) -> Dict[str, Any]:  # Returns organization statistics
    """
    Organize images into folders based on their anomaly scores.
    
    Args:
        prediction_results: List of prediction results, each containing 'image_path' and 'anomaly_score'
        output_dir: Base directory where score folders will be created
        score_thresholds: List of threshold values (e.g., [0.5, 1.0] for simple two-folder setup)
        copy_mode: Whether to copy (True) or move (False) images
        save_metadata: Whether to save JSON metadata for each folder
    
    Returns:
        Dictionary with organization statistics
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    print("\n🗂️  ORGANIZING IMAGES BY ANOMALY SCORE")
    print("="*70)
    print(f"📂 Output directory: {output_dir}")
    print(f"📊 Score thresholds: {score_thresholds}")
    print(f"📋 Total images: {len(prediction_results)}")
    print(f"🔄 Mode: {'COPY' if copy_mode else 'MOVE'}")
    
    # Create score folders
    folder_map = create_score_folders(output_dir, score_thresholds)
    
    # Track statistics
    folder_stats = {str(t): {'count': 0, 'images': [], 'scores': []} for t in score_thresholds}
    failed_count = 0
    
    print("\n📦 Processing images...")
    
    # Process each image
    for result in tqdm(prediction_results, desc="Organizing images"):
        try:
            image_path = result.get('image_path')
            anomaly_score = result.get('anomaly_score')
            
            if image_path is None or anomaly_score is None:
                print(f"⚠️  Skipping result with missing data: {result}")
                failed_count += 1
                continue
            
            # Save image to appropriate folder
            dest_path = save_image_by_score(
                image_path=image_path,
                anomaly_score=anomaly_score,
                output_dir=output_dir,
                score_thresholds=score_thresholds,
                copy_mode=copy_mode
            )
            
            # Update statistics
            folder_name = determine_score_folder(anomaly_score, score_thresholds)
            folder_stats[folder_name]['count'] += 1
            folder_stats[folder_name]['images'].append(str(dest_path))
            folder_stats[folder_name]['scores'].append(float(anomaly_score))
            
        except Exception as e:
            print(f"❌ Error processing {result.get('image_path', 'unknown')}: {e}")
            failed_count += 1
    
    # Save metadata for each folder if requested
    if save_metadata:
        print("\n💾 Saving metadata...")
        for folder_name, stats in folder_stats.items():
            if stats['count'] > 0:
                metadata_path = folder_map[folder_name] / "metadata.json"
                metadata = {
                    'threshold': folder_name,
                    'count': stats['count'],
                    'avg_score': float(np.mean(stats['scores'])) if stats['scores'] else 0.0,
                    'min_score': float(np.min(stats['scores'])) if stats['scores'] else 0.0,
                    'max_score': float(np.max(stats['scores'])) if stats['scores'] else 0.0,
                    'images': stats['images']
                }
                with open(metadata_path, 'w') as f:
                    json.dump(metadata, f, indent=2)
    
    # Print summary
    print("\n📊 ORGANIZATION SUMMARY")
    print("="*70)
    for threshold in sorted(score_thresholds):
        folder_name = str(threshold)
        count = folder_stats[folder_name]['count']
        if count > 0:
            avg_score = np.mean(folder_stats[folder_name]['scores'])
            print(f"📁 Folder '{folder_name}': {count} images (avg score: {avg_score:.4f})")
    
    if failed_count > 0:
        print(f"\n⚠️  Failed: {failed_count} images")
    
    print("\n✅ Organization complete!")
    
    return {
        'output_dir': str(output_dir),
        'score_thresholds': score_thresholds,
        'folder_stats': {k: {'count': v['count'], 
                             'avg_score': float(np.mean(v['scores'])) if v['scores'] else 0.0}
                        for k, v in folder_stats.items()},
        'total_processed': len(prediction_results) - failed_count,
        'failed_count': failed_count
    }

## High-Level Workflow Function

In [ ]:
#| export
def predict_and_organize_by_score(
    model_path: Union[str, Path],  # Path to the trained model
    image_list_file: Union[str, Path],  # Text file with image paths (one per line)
    output_dir: Union[str, Path],  # Base output directory for organized images
    score_thresholds: List[float] = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Score thresholds
    batch_id: Optional[str] = None,  # Optional batch identifier
    copy_mode: bool = True,  # If True, copy files; if False, move files
    save_metadata: bool = True,  # If True, save metadata JSON for each folder
    device: str = "auto",  # Device for inference ("auto", "cpu", "cuda")
    **kwargs  # Additional arguments passed to prediction function
) -> Dict[str, Any]:  # Returns combined prediction and organization results
    """
    Complete workflow: Predict anomaly scores and organize images into score-based folders.
    
    This is the main function that combines:
    1. Smart batch creation
    2. Prediction using predict_image_list_from_file_enhanced
    3. Image organization based on anomaly scores
    
    Args:
        model_path: Path to the trained anomaly detection model
        image_list_file: Text file containing paths to images (one per line)
        output_dir: Directory where score-based folders will be created
        score_thresholds: List of threshold values (customize to your needs)
            Examples:
            - [0.5, 1.0] for simple two-folder setup
            - [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0] for fine-grained organization
        batch_id: Optional identifier for this batch
        copy_mode: Whether to copy (True) or move (False) images
        save_metadata: Whether to save JSON metadata for each folder
        device: Device to use for inference
        **kwargs: Additional arguments (save_heatmap, heatmap_style, etc.)
    
    Returns:
        Dictionary containing:
        - prediction_results: Full prediction results
        - organization_stats: Statistics about image organization
    """
    print("\n🚀 PREDICT AND ORGANIZE BY ANOMALY SCORE")
    print("="*70)
    
    # Step 1: Run predictions
    print("\n📊 Step 1: Running predictions...")
    prediction_output = predict_image_list_from_file_enhanced(
        model_path=model_path,
        image_list_file=image_list_file,
        batch_id=batch_id,
        output_dir=output_dir,
        device=device,
        save_results=True,
        **kwargs
    )
    
    # Extract results
    prediction_results = prediction_output.get('results', [])
    
    if not prediction_results:
        print("⚠️  No prediction results to organize!")
        return {
            'prediction_results': prediction_output,
            'organization_stats': None
        }
    
    print(f"✅ Predictions complete: {len(prediction_results)} images processed")
    
    # Step 2: Organize images by score
    print("\n📁 Step 2: Organizing images by score...")
    organization_stats = organize_images_by_score(
        prediction_results=prediction_results,
        output_dir=output_dir,
        score_thresholds=score_thresholds,
        copy_mode=copy_mode,
        save_metadata=save_metadata
    )
    
    print("\n🎉 WORKFLOW COMPLETE!")
    print("="*70)
    
    return {
        'prediction_results': prediction_output,
        'organization_stats': organization_stats
    }

## Example Usage

```python
# Example 1: Simple two-folder organization (low vs high anomaly)
results = predict_and_organize_by_score(
    model_path="path/to/model.ckpt",
    image_list_file="path/to/images.txt",
    output_dir="organized_output",
    score_thresholds=[0.5, 1.0],  # Two folders: 0.5 (normal) and 1.0 (anomaly)
    copy_mode=True
)

# Example 2: Fine-grained organization with 8 score folders
results = predict_and_organize_by_score(
    model_path="path/to/model.ckpt",
    image_list_file="path/to/images.txt",
    output_dir="organized_output",
    score_thresholds=[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    copy_mode=True,
    save_heatmap=True,
    heatmap_style="side_by_side"
)

# Example 3: Custom thresholds
results = predict_and_organize_by_score(
    model_path="path/to/model.ckpt",
    image_list_file="path/to/images.txt",
    output_dir="organized_output",
    score_thresholds=[0.25, 0.5, 0.75, 1.0],  # Four folders
    copy_mode=False  # Move files instead of copying
)
```

## Tests

In [ ]:
#| hide
# Test determine_score_folder
test_eq(determine_score_folder(0.3, [0.5, 1.0]), "0.5")
test_eq(determine_score_folder(0.7, [0.5, 1.0]), "1.0")
test_eq(determine_score_folder(0.45, [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]), "0.5")
test_eq(determine_score_folder(0.85, [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]), "0.9")
print("✅ All tests passed!")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()